# Agent Skills Example (PandasAI v3.0.0)

This notebook demonstrates how to use custom skills with an `Agent` in PandasAI v3.0.0.
Skills are automatically registered when you use the `@pai.skill` decorator, which adds custom functions (skills) globally. No need to manually add skills to the agent - they are available automatically.

In [1]:
import os
from pandasai_litellm.litellm import LiteLLM
from pandasai import Agent
import pandasai as pai
import pandas as pd

# 1. Configure LLM and PandasAI
api_key = os.getenv("OPENAI_API_KEY", "your-api-key")
llm = LiteLLM(model="gpt-5-mini", api_key=api_key)
pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})

# 2. Prepare DataFrame

payroll_raw = pd.DataFrame([
    {"employee": "Alice", "role": "AE", "salary": 90000,
        "performance": 0.92, "region": "US"},
    {"employee": "Boris", "role": "AE", "salary": 85000,
        "performance": 0.88, "region": "EU"},
    {"employee": "Chen", "role": "CSM", "salary": 78000,
        "performance": 0.95, "region": "APAC"},
])
payroll_semantic = pai.DataFrame(
    payroll_raw,
    name="employee_payroll",
    description="Compensation and performance metrics",
)


# 4. Initialize Agent (skills are automatically available)
agent = Agent([payroll_semantic])




In [2]:
original_query = "Show me the total salary for each region"
print(f"Original query: {original_query}")

# Option 1: Use chat() directly with the original query (recommended)
print("\n--- Option 1: Direct query execution ---")
response = agent.chat(original_query)
print(f"Response: {response}")

# Option 2: Use chat() to get a rephrased version of the query
print("\n--- Option 2: Query rephrasing via chat() ---")
rephrase_prompt = f"Rephrase this query to be more clear and concise: '{original_query}'. Return only the rephrased query, nothing else."
rephrased_query = agent.chat(rephrase_prompt)
print(f"Rephrased query: {rephrased_query}")

Original query: Show me the total salary for each region

--- Option 1: Direct query execution ---
Response:   region  total_salary
0     US       90000.0
1     EU       85000.0
2   APAC       78000.0

--- Option 2: Query rephrasing via chat() ---
Rephrased query: Total salary by region


In [3]:
def clarification_questions(smart_object, query: str):
    return smart_object.chat(f"Какие уточнения нужны для: {query}?")


def explain(smart_object, query: str):
    return smart_object.chat(f"Объясни шаги расчёта для: {query}")


def follow_up(smart_object, query: str):
    return smart_object.chat(f"Следующий вопрос: {query}")


def rephrase_query(smart_object, query: str):
    return smart_object.chat(f"Переформулируй запрос: {query}")


# Примеры использования с payroll данными
clarification_questions(agent, "Как рассчитывается средняя зарплата?")
explain(agent, "Почему зарплаты отличаются по регионам?")
follow_up(agent, "А как связана зарплата с производительностью?")
rephrase_query(agent, "Сравни зарплаты в EU и US")

clarification_questions(agent, "Дай больше деталей о зарплатах по ролям")


StringResponse(type='string', value='Чтобы подготовить детальный и корректный анализ зарплат по ролям, нужны следующие уточнения:\n\n1) Желательная гранулярность:\n- Нужна ли разбивка по регионам/странам (например, US / EU / APAC)? (выполнена сводка по роли+региону)\n- Нужна ли детализация по seniority/уровню (junior / mid / senior / lead) внутри ролей?\n\n2) Какие компоненты компенсации включать:\n- Только базовая зарплата или полная компенсация (бонусы, опционы, компенсации в других валютах)?\n- Нужно ли нормализовать валюты / пересчитывать в одну валюту?\n\n3) Метрики и визуализация:\n- Какие метрики важны: среднее, медиана, стандартное отклонение, квантили (P10/P25/P75/P90), диапазон, выбросы?\n- Нужны ли графики (ящик с усами/гистограммы/распределение/boxplot) или достаточно таблиц?\n\n4) Фильтрация и качество данных:\n- Период (например, текущий год, последние 12 мес)?\n- Исключать ли сотрудников с неполными данными или аномальными значениями? Какой порог минимального N для показ

In [4]:
def enable_agent_extras(agent_instance):
    def _clarification(prompt: str):
        return agent_instance.chat(f"Сформулируй уточняющие вопросы к запросу: {prompt}")

    def _explain(prompt: str):
        return agent_instance.chat(f"Объясни шаги решения для: {prompt}")

    def _rephrase(prompt: str):
        return agent_instance.chat(f"Переформулируй запрос более кратко: {prompt}")

    agent_instance.clarification_questions = _clarification
    agent_instance.explain = _explain
    agent_instance.rephrase_query = _rephrase
    return agent_instance


agent = enable_agent_extras(agent)
# Примеры использования с payroll данными
agent.clarification_questions("Сводка по зарплатам")
agent.explain("Почему зарплаты отличаются по ролям")
agent.rephrase_query("Покажи среднюю зарплату по регионам")


DataFrameResponse(type='dataframe', value=  region  n  avg_salary
0     US  1     90000.0
1     EU  1     85000.0
2   APAC  1     78000.0)